In [ ]:
import numpy as np, torch, time, os
from Ctubes.plot_utils import plot_generatrix
from Ctubes.tubes import Directrix, Generatrix, CTube
from Ctubes.opt import CTubeOptimizationProblem
from Ctubes.misc_utils import get_pairings_all, load_curve_from_obj
from Ctubes.path_utils import get_name, setup_paths

paths = setup_paths(get_name())

PI = np.pi
TORCH_DTYPE = torch.float64
torch.set_default_dtype(TORCH_DTYPE)
torch.set_printoptions(precision=4)

%load_ext autoreload
%autoreload 2

%matplotlib widget

# Figure 2: C-tube Mold for Casting

In [ ]:
# Directrix: read curve from file
cps_ref = load_curve_from_obj(os.path.join(paths["data"], "fig2_casting/directrix_Q.obj"))
K = cps_ref.shape[0]
M = (K - 1) * 5 + 1

In [ ]:
# Generatrix: rhombus
N = 4
tube_radius = 0.8

generatrix_2d = torch.zeros(size=(N, 2))
generatrix_2d[0, 0] =   tube_radius * 1.0
generatrix_2d[1, 1] =   tube_radius * torch.sqrt(torch.tensor(3.0))
generatrix_2d[2, 0] = - tube_radius * 1.0
generatrix_2d[3, 1] = - tube_radius * torch.sqrt(torch.tensor(3.0))

fig, ax = plot_generatrix(generatrix_2d)

In [ ]:
# C-tube
directrix = Directrix(cps_ref, M)

X0 = directrix.X[0]
T0 = directrix.get_tangents()[0]
generatrix = Generatrix(generatrix_2d, X0, T0)

tube = CTube(directrix, generatrix)

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
# Load target cross-section radii from file
target_cross_section_radii = torch.tensor(
    np.loadtxt(os.path.join(paths["data"], "fig2_casting/target_cross_section_radii.txt")), 
    dtype=TORCH_DTYPE
)

In [ ]:
# Set up optimization problem

opt_weights = {
    'match_target_cross_section_radius': 1.6e1 / generatrix.aabb_diagonal_length() ** 2,
    'smooth_plane_normal_diffs': 1e-1,
    'smooth_apex_loc_func': 1e2,
    'join_ends': 1e1 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'target_cross_section_radii': target_cross_section_radii,
    'join_ends_pairings': get_pairings_all(N),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)

opt_prob.activate_cps(False)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(True)
opt_prob.activate_plane_normals(True)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []  # no fixed DOF by default

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

In [ ]:
fig, ax = opt_prob.plot_cross_section_radii()